### **Probemos**

In [5]:

## Librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd
from tqdm import tqdm  # pip install tqdm
import datetime as dt
import re, time
import json

In [11]:

## Extraccion de Informacion de segun el tipo de Vehicuklo y Condicion

def get_user_choice(prompt, choices):
    """Función para obtener una elección válida del usuario a partir de un diccionario."""
    while True:
        print(prompt)
        for key, (desc, val) in choices.items():
            print(f"  {desc} ({key})")
        choice = input("Su elección: ")
        if choice in choices:
            return choices[choice][1]
        else:
            print("\nOpción no válida. Por favor, intente de nuevo.\n")

# Definimos las opciones para que sea más fácil de mantener
TIPO_VEHICULO_CHOICES = {
    "1": ("autos", "autos"),
    "2": ("motos", "motos"),
    "3": ("camiones", "camiones")
}
CATEGORIA_CHOICES = {
    "1": ("camioneta", "camionetas-suv"),
    "2": ("sedan", "sedan"),
    "3": ("hatchback", "hatchback"),
    "4": ("pick-up", "pick-up"),
    "5": ("vans", "vans"),
    "6": ("deportivo", "deportivo")
}
UNIDAD_CHOICES = {
    "1": ("nuevo", "nuevos"),
    "2": ("usados", "usados"),
    "3": ("seminuevos", "seminuevos")
}

tipo_vehiculo = get_user_choice("¿Qué tipo de unidad quiere extraer?", TIPO_VEHICULO_CHOICES)
categoria = get_user_choice("¿Qué categoría de auto desea?", CATEGORIA_CHOICES)
unidad = get_user_choice("¿Qué condición de unidad busca?", UNIDAD_CHOICES)

print(f"\nSe extraerán datos de: {tipo_vehiculo} - {categoria}- {unidad}")

print("\nGenerando URLs a extraer...")

url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}--{categoria}"


print(url)


¿Qué tipo de unidad quiere extraer?
  autos (1)
  motos (2)
  camiones (3)
¿Qué categoría de auto desea?
  camioneta (1)
  sedan (2)
  hatchback (3)
  pick-up (4)
  vans (5)
  deportivo (6)
¿Qué condición de unidad busca?
  nuevo (1)
  usados (2)
  seminuevos (3)

Se extraerán datos de: autos - camionetas-suv- seminuevos

Generando URLs a extraer...
https://neoauto.com/venta-de-autos-seminuevos--camionetas-suv


In [12]:

options = webdriver.ChromeOptions()
# options.add_argument("--headless")   # modo oculto
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
driver.get(url)

resultados = driver.find_element(By.CLASS_NAME, "s-results__count")

paginas_consulta = int(re.sub(r'[^\d]', '', resultados.text.split()[0]))

print("\nNumero de Anuncios a extraer:", paginas_consulta)

numero_paginas = (paginas_consulta // 20) + 1

print("\nNumero de paginas a extraer:", numero_paginas)


Numero de Anuncios a extraer: 545

Numero de paginas a extraer: 28


In [13]:
paginas = []

# Este bucle debe ejecutarse siempre, no solo para "usados".
for i in  range(1, numero_paginas+1):
    url_pagina = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}--{categoria}?page={i}"
    paginas.append({
        "url": url_pagina,
        "tipo": unidad,
        "tipo_vehiculo": tipo_vehiculo,
        "categoria": categoria
    })

print("\nInicio de Extraccion de datos...")

# --- Config mínimo ---
YEAR_RX  = re.compile(r'\b(19[8-9]\d|20[0-3]\d)\b')
NOISE_RX = re.compile(r'\bauto\s+nuev[oa]s?\b', re.I)
BRANDS = sorted([
        # Europa
        "Alfa Romeo","Aston Martin","Audi","Bentley","BMW","Bugatti","Citroen","Cupra",
        "Dacia","Ferrari","Fiat","Jaguar","Lamborghini","Land Rover","Maserati","Maybach",
        "McLaren","Mercedes Benz","Mini","Opel","Peugeot","Porsche","Renault","Rolls Royce",
        "Seat","Skoda","Smart","Volkswagen","Volvo",
    
        # Japón
        "Acura","Daihatsu","Honda","Infiniti","Isuzu","Lexus","Mazda","Mitsubishi",
        "Nissan","Subaru","Suzuki","Toyota",
    
        # Corea
        "Hyundai","Genesis","Kia","SsangYong",
    
        # Estados Unidos
        "Buick","Cadillac","Chevrolet","Chrysler","Dodge","Ford","GMC","Hummer","Jeep",
        "Lincoln","Ram","Tesla",
    
        # China (muy usados en Latam)
        "BAIC","BYD","Changan","Chery","DFSK","Dongfeng","FAW","Foton","Geely","Great Wall",
        "Haval","JAC","Jetour","Maxus","MG","Omoda","Zotye", "JMC",
    
        # India
        "Mahindra","Tata",
    
        # Reino Unido
        "Lotus","Morgan","Noble","Vauxhall",
    
        # Otros/Exóticos
        "Abarth","Apollo","Arash","Arrinera","Artega","Brilliance","Caterham","DeLorean",
        "Denza","Fisker","Koenigsegg","Lucid","Pagani","Polestar","Rimac","Rover","Saab",
        "Scion","Spyker","Weismann","Zenvo"
    ], key=len, reverse=True)

def clean_title(s):  # quita "AUTO NUEVO", espacios dobles
    return re.sub(r'\s+', ' ', NOISE_RX.sub('', s)).strip()

def parse_brand_model_year(t):
    t = clean_title(t)
    # año: último match si hay varios
    ys = YEAR_RX.findall(t); anio = ys[-1] if ys else None
    # marca: primera que aparezca (BRANDS ya está ordenado por longitud desc)
    tl = t.lower()
    marca = next((b for b in BRANDS if b.lower() in tl), "Otra Marca")
    # modelo: t sin marca ni año
    modelo = re.sub(re.escape(marca), '', t, flags=re.I).strip() if marca != "Otra Marca" else t
    if anio: modelo = re.sub(rf'\b{anio}\b', '', modelo).strip()
    return t, marca, modelo, anio

neo_autos = []

for pagina in paginas[:1]:  # pagina = {"url": ..., "tipo": ..., "tipo_vehiculo": ...}
    driver.get(pagina["url"])

    # 1) espera a que aparezcan resultados
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.c-results__link")))

    # 2) scroll simple (3–4 bajadas) para forzar lazy-load
    prev = 0
    for _ in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.2)
        n = len(driver.find_elements(By.CSS_SELECTOR, "a.c-results__link"))
        if n == prev: break
        prev = n

    # 3) recorre por tarjeta (evita desalineaciones)
    cards = driver.find_elements(By.CSS_SELECTOR, "a.c-results__link")
    vistos = set()

    for a in cards:
        url = a.get_attribute("href")
        if not url or url in vistos: 
            continue
        vistos.add(url)

        # sube al contenedor de la card (si falla, usamos el link como fallback)
        try:
            card = a.find_element(By.XPATH, "./ancestor::*[self::li or self::article][1]")
        except:
            card = a

        # título
        try:
            titulo_raw = card.find_element(By.CSS_SELECTOR, ".c-results__header").text
        except:
            titulo_raw = ""

        titulo, marca, modelo, anio = parse_brand_model_year(titulo_raw)

        # precio (si no hay, None)
        try:
            precio = card.find_element(By.CSS_SELECTOR, ".c-results-mount__price").text

        except:
            precio = None

        # detalle (opcional)
        try:
            detalle = card.find_element(By.CSS_SELECTOR, ".c-results-details__description").text

            # normaliza y separa líneas no vacías
            t = re.sub(r'[\r\xa0]', ' ', detalle).strip()
            lines = [l.strip() for l in t.splitlines() if l.strip()]

            # 1) combustible y transmisión (vienen en la 1ra línea separados por "|")
            combustible = transmision_raw = None
            if lines:
                p0 = re.split(r'\s*\|\s*', lines[0])  # acepta con o sin espacios: "Gasolina|Automática - Secuencial"
                combustible = p0[0].title() if p0 else None
                transmision_raw = p0[1] if len(p0) > 1 else None

            # 1.1) separa transmisión en tipo y caja si viene "Automática - Secuencial"
            tipo_transmision = caja = None
            if transmision_raw:
                parts = [p.strip() for p in transmision_raw.split("-")]
                tipo_transmision = parts[0]
                caja = parts[1] if len(parts) > 1 else None

            # 2) kilometraje (busca en todo el texto)
            km_m = re.search(r'(\d[\d.,]*)\s*kms?', t, flags=re.I)
            kilometraje_km = int(re.sub(r'[^\d]', '', km_m.group(1))) if km_m else None

            # 3) ubicación (primera línea con coma que no sea "Kms")
            ubicacion = next((l for l in lines if ',' in l and 'km' not in l.lower()), None)
            
        except:
            detalle = ""
            

        # tags (opcionales)
        tags = [e.text.strip() for e in card.find_elements(By.CSS_SELECTOR, ".c-results-tag__stick") if e.text.strip()]

        if len(tags) == 1:
            tags = tags[0]
            
        img_urls = []
        imgs = card.find_elements(By.CSS_SELECTOR, "img.c-results-slider__img-inside, img.c-results__image, img.c-results-slider__img")  # varios selectores por si cambian

        for im in imgs:
            try:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", im)
            except:
                pass
            time.sleep(0.05)
            u = im.get_attribute("data-src") or im.get_attribute("src")
            if not u:
                srcset = im.get_attribute("data-srcset") or im.get_attribute("srcset")
                if srcset:
                    u = srcset.split(",")[0].strip().split()[0]
            if u and u not in img_urls:
                img_urls.append(u)

        img_url_primaria = img_urls[0] if img_urls else None
        
        neo_autos.append({
            "titulo": titulo,
            "tipo_vehiculo": pagina["tipo_vehiculo"],
            "categoria": pagina["categoria"],
            "marca": marca,
            "modelo": modelo,
            "año": anio,
            "precio": precio,
            "detalle": detalle,
            "combustible": combustible,
            "tipo_transmision": tipo_transmision,
            "caja": caja,
            "kilometraje_km": kilometraje_km,
            "ubicacion": ubicacion, 
            "tags": tags,
            "url_auto": url,
            "tipo": pagina["tipo"],
            "pagina": pagina["url"],
            ## Imagenes
            "img_urls": img_urls,
        })


Inicio de Extraccion de datos...


## **Extraccion de imagenes - Neo Autos V4**

In [14]:

# CDN_HINT = "cdn.neoauto.com/elements/autos"  ## caso seminuevos o usados
# CDN_HINT = "cdn.neoauto.com/elements/nuevo" ## caso nuevos


if unidad == "nuevos":
    CDN_HINT = "cdn.neoauto.com/elements/nuevo"
else:
    CDN_HINT = "cdn.neoauto.com/elements/autos"

def _best_from_srcset(srcset: str) -> str | None:
    if not srcset:
        return None
    cand = []
    for part in srcset.split(","):
        bits = part.strip().split()
        if not bits:
            continue
        url = bits[0]
        w = 0
        if len(bits) > 1 and bits[1].endswith("w"):
            try:
                w = int(re.sub(r"\D", "", bits[1]))
            except:
                w = 0
        cand.append((w, url))
    return max(cand)[1] if cand else None

def get_image_urls(driver, timeout=5):
    wait = WebDriverWait(driver, timeout)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    urls = set()

    # --------- A) Intento 1: modal abierto (slick-list con "Foto Galeria") ----------
    modal_selectors = [
        "div[role='dialog'] .slick-list picture img",
        "div[aria-modal='true'] .slick-list picture img",
        "div[role='dialog'] .slick-list picture source",
        "div[aria-modal='true'] .slick-list picture source",
        "div[role='dialog'] .slick-list picture[style*='background-image']",
        "div[aria-modal='true'] .slick-list picture[style*='background-image']",
    ]
    for sel in modal_selectors:
        for el in driver.find_elements(By.CSS_SELECTOR, sel):
            tag = el.tag_name.lower()
            if tag == "img":
                hi = _best_from_srcset(el.get_attribute("srcset") or "")
                src = hi or (el.get_attribute("src") or "")
                if CDN_HINT in src:
                    urls.add(src)
                ds = el.get_attribute("data-src") or ""
                if CDN_HINT in ds:
                    urls.add(ds)
            elif tag == "source":
                ss = el.get_attribute("srcset") or ""
                best = _best_from_srcset(ss)
                if best and CDN_HINT in best:
                    urls.add(best)
            else:  # picture con background-image
                style = el.get_attribute("style") or ""
                m = re.search(r"url\(['\"]?(.*?)['\"]?\)", style)
                if m and CDN_HINT in m.group(1):
                    urls.add(m.group(1))

    if urls:
        return sorted(urls)

    # --------- B) Intento 2: ficha (grid de 2 columnas con miniaturas) -------------
    # mueve a la galería para forzar lazy-load
    try:
        gallery = wait.until(EC.presence_of_element_located(
            (By.XPATH, "(//div[contains(@class,'grid') and contains(@class,'grid-cols-2')])[1]")
        ))
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", gallery)
        time.sleep(0.8)
    except Exception:
        pass

    ficha_selectors = [
        ".grid.grid-cols-2 picture img",
        ".grid.grid-cols-2 picture source",
        ".grid.grid-cols-2 picture[style*='background-image']",
        # comodín: cualquier img del CDN en la página
        f"img[src*='{CDN_HINT}']"
    ]
    for sel in ficha_selectors:
        for el in driver.find_elements(By.CSS_SELECTOR, sel):
            tag = el.tag_name.lower()
            if tag == "img":
                hi = _best_from_srcset(el.get_attribute("srcset") or "")
                src = hi or (el.get_attribute("src") or "")
                if CDN_HINT in src:
                    urls.add(src)
                ds = el.get_attribute("data-src") or ""
                if CDN_HINT in ds:
                    urls.add(ds)
            elif tag == "source":
                ss = el.get_attribute("srcset") or ""
                best = _best_from_srcset(ss)
                if best and CDN_HINT in best:
                    urls.add(best)
            else:
                style = el.get_attribute("style") or ""
                m = re.search(r"url\(['\"]?(.*?)['\"]?\)", style)
                if m and CDN_HINT in m.group(1):
                    urls.add(m.group(1))

    return sorted(urls)

In [15]:
enlace_anuncio = [{"url_auto": auto["url_auto"], "titulo": auto["titulo"], "tipo_vehiculo": auto["tipo"]} for auto in neo_autos]
enlace_anuncio

[{'url_auto': 'https://neoauto.com/auto/seminuevo/bmw-x6-30d-2016-1851383',
  'titulo': 'Bmw X6 30d 2016',
  'tipo_vehiculo': 'seminuevos'},
 {'url_auto': 'https://neoauto.com/auto/seminuevo/land-rover-range-rover-evoque-2016-1840476',
  'titulo': 'Land Rover Range Rover Evoque 2016',
  'tipo_vehiculo': 'seminuevos'},
 {'url_auto': 'https://neoauto.com/auto/seminuevo/bmw-x3-2021-1851433',
  'titulo': 'Bmw X3 2021',
  'tipo_vehiculo': 'seminuevos'},
 {'url_auto': 'https://neoauto.com/auto/seminuevo/porsche-macan-2024-1853919',
  'titulo': 'Porsche Macan 2024',
  'tipo_vehiculo': 'seminuevos'},
 {'url_auto': 'https://neoauto.com/auto/seminuevo/toyota-rav4-2022-1845323',
  'titulo': 'Toyota Rav4 2022',
  'tipo_vehiculo': 'seminuevos'},
 {'url_auto': 'https://neoauto.com/auto/seminuevo/volvo-xc-40-2023-1851654',
  'titulo': 'Volvo Xc-40 2023',
  'tipo_vehiculo': 'seminuevos'},
 {'url_auto': 'https://neoauto.com/auto/seminuevo/land-rover-range-rover-sport-2020-1838557',
  'titulo': 'Land Ro

In [ ]:

enlace_anuncio = [{"url_auto": auto["url_auto"]
                , "titulo": auto["titulo"]
                , "tipo_vehiculo": auto["tipo"]} for auto in neo_autos]

# options = webdriver.ChromeOptions()
# # options.add_argument("--headless")   # modo oculto
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# # options.add_experimental_option("excludeSwitches", ["enable-automation"])
# # options.add_experimental_option('useAutomationExtension', False)
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)

W = WebDriverWait(driver, 5)

FIN_PATTERNS = (
    "anuncio finalizado", "aviso finalizado",
    "ya no está disponible", "ya no existe",
    "aviso no disponible", "no disponible", "no encontrado"
)

def is_finalizado(driver) -> bool:
    try:
        body = W.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        txt = (body.get_attribute("innerText") or "").lower()
        return any(p in txt for p in FIN_PATTERNS)
    except TimeoutException:
        return False

def get_descripcion(driver):
    """
    Extrae la descripción de un anuncio (nuevo o seminuevo).
    Busca en diferentes estructuras HTML posibles.
    """
    selectores = [
        # Caso común en autos seminuevos
        (By.XPATH, "//section[@id='Descripción']//div[contains(@class,'whitespace-pre-wrap')]"),
        # Caso general o fallback
        (By.XPATH, "//h2[contains(.,'Descripción')]/following::div[contains(@class,'whitespace-pre-wrap')][1]"),
        # Caso autos nuevos (como en tu captura)
        (By.CSS_SELECTOR, "div.w-full.whitespace-pre-wrap.break-words.text-justify.font-ubuntu.text-base.text-black"),
        # Variante más laxa (por si cambian clases)
        (By.CSS_SELECTOR, "div[class*='whitespace-pre-wrap'][class*='break-words'][class*='text-justify']"),
    ]

    for by, selector in selectores:
        try:
            elems = driver.find_elements(by, selector)
            if elems:
                texto = elems[0].text.strip()
                if texto:
                    return texto
        except Exception:
            continue

    return None


def get_especificaciones(driver):
    """Busca el primer grid label→valor; si no existe devuelve {}."""
    grid = driver.find_element(By.CSS_SELECTOR, "div.grid.grid-cols-2")

    # Obtener todas las celdas (div hijos)
    cells = grid.find_elements(By.XPATH, "./div")

    # Armar diccionario con pares
    specs = {}
    for i in range(0, len(cells), 2):
        key = cells[i].text.strip()
        val = cells[i+1].text.strip()
        specs[key] = val
        
    return specs

In [39]:
for enlace_i in enlace_anuncio[:3]:
    
    url = enlace_i["url_auto"]

    driver.get(url)
    W.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(0.4)

In [23]:
descripcion_element = driver.find_element(By.CSS_SELECTOR, "div.flex.w-full.flex-col.gap-3xlarge.whitespace-pre-wrap.break-words")
descripcion_texto = descripcion_element.text

print("Descripción extraída:\n", descripcion_texto)

Descripción extraída:
 BMW X2 SDRIVE20I 4X2 2021 Placa IMPAR. Color GRIS. UNIDAD UBICADA EN: Av. La Marina 3386, San Miguel (DIVEUSADOS DIVEMOTOR)/ Financiamiento a través de crédito vehicular con entidades bancarias (previa evaluación crediticia del cliente).


In [22]:
for enlace_i in enlace_anuncio[:3]:
    
    url = enlace_i["url_auto"]

    driver.get(url)
    W.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(0.4)
    
    # descripcion = get_descripcion(driver)
    
    imagenes = get_image_urls(driver)
    
    print(imagenes)

['https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_2109266.webp', 'https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_2594474.webp', 'https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_3634362.webp', 'https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_4528856.webp', 'https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_7019794.webp']
['https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_1878428.webp', 'https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_2128287.webp', 'https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_5739076.webp', 'https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_6517569.webp', 'https://cdn.neoauto.com/elements/autos_usados/960x584/113873/113873_7771183.webp']
['https://cdn.neoauto.com/elements/autos_usados/960x584/746429/746429_3967236.webp', 'https://cdn.neoauto.com/elements/autos_usados/960x584/746429/746429_4197

In [18]:

# --- loop principal
neo_autos_img = []
for enlace_i in enlace_anuncio[:10]:
    
    url = enlace_i["url_auto"]
    tipo = enlace_i.get("tipo_vehiculo", "").lower()
    
    try:
        driver.get(url)
        W.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(0.4)

        if is_finalizado(driver):
            neo_autos_img.append({
                "titulo": enlace_i.get("titulo"),
                "tipo_vehiculo": enlace_i.get("tipo_vehiculo"),
                "url_auto": url,
                "estado": "finalizado",
                "descripcion": None,
                "especificaciones": {},
                "imagenes": []
            })
            continue

        # activo → extraemos lo que haya
        descripcion = get_descripcion(driver)
        
        # Especificaciones → solo si es seminuevo o usado
        if tipo in ["seminuevos", "usados"]:
            especificaciones = get_especificaciones(driver)
        else:
            especificaciones = {}

        # si usas tu función de imágenes:
        try:
            # imagenes = get_image_urls(driver, open_modal=False)
            imagenes = get_image_urls(driver)
        except Exception:
            imagenes = []

        neo_autos_img.append({
            "titulo": enlace_i.get("titulo"),
            "tipo_vehiculo": enlace_i.get("tipo_vehiculo"),
            "url_auto": url,
            "estado": "ok",
            "descripcion": descripcion,
            "especificaciones": especificaciones,
            "imagenes": imagenes
        })

    except Exception as e:
        neo_autos_img.append({
            "titulo": enlace_i.get("titulo"),
            "tipo_vehiculo": tipo,
            "url_auto": url,
            "estado": "error",
            "error_msg": str(e),
            "descripcion": None,
            "especificaciones": {},
            "imagenes": []
        })

In [20]:
neo_autos[:10]

[{'titulo': 'Bmw X6 30d 2016',
  'tipo_vehiculo': 'autos',
  'categoria': 'camionetas-suv',
  'marca': 'BMW',
  'modelo': 'X6 30d',
  'año': '2016',
  'precio': 'US$ 55,000',
  'detalle': 'Diesel|Automática - Secuencial\nKilometraje:\n44,626 Kms\nLima, Lima\nBMW X6 M 30D XDRIVE',
  'combustible': 'Diesel',
  'tipo_transmision': 'Automática',
  'caja': 'Secuencial',
  'kilometraje_km': 44626,
  'ubicacion': 'Lima, Lima',
  'tags': 'DE LUJO',
  'url_auto': 'https://neoauto.com/auto/seminuevo/bmw-x6-30d-2016-1851383',
  'tipo': 'seminuevos',
  'pagina': 'https://neoauto.com/venta-de-autos-seminuevos--camionetas-suv?page=1',
  'img_urls': ['https://cdn.neoauto.com/elements/autos_usados/196x165/113873/113873_2639329.webp',
   'https://cdn.neoauto.com/elements/autos_usados/196x165/113873/113873_3289952.webp',
   'https://cdn.neoauto.com/elements/autos_usados/196x165/113873/113873_4528856.webp',
   'https://cdn.neoauto.com/elements/autos_usados/196x165/113873/113873_7019794.webp',
   'https:/

In [21]:
neo_autos_img

[{'titulo': 'Bmw X6 30d 2016',
  'tipo_vehiculo': 'seminuevos',
  'url_auto': 'https://neoauto.com/auto/seminuevo/bmw-x6-30d-2016-1851383',
  'estado': 'error',
  'error_msg': 'Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.grid.grid-cols-2"}\n  (Session info: chrome=141.0.7390.108); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception\nStacktrace:\n\tGetHandleVerifier [0x0x64fe83+66515]\n\tGetHandleVerifier [0x0x64fec4+66580]\n\t(No symbol) [0x0x43dc48]\n\t(No symbol) [0x0x488704]\n\t(No symbol) [0x0x488aab]\n\t(No symbol) [0x0x4cf482]\n\t(No symbol) [0x0x4ab214]\n\t(No symbol) [0x0x4ccba7]\n\t(No symbol) [0x0x4aafc6]\n\t(No symbol) [0x0x47c2ca]\n\t(No symbol) [0x0x47d154]\n\tGetHandleVerifier [0x0x8a73b3+2521347]\n\tGetHandleVerifier [0x0x8a2333+2500739]\n\tGetHandleVerifier [0x0x677cd4+229924]\n\tGetHandleVerifier [0x0x668238+165768]\n\tGetHandleV